<a href="https://colab.research.google.com/github/someDeveloperDH/game/blob/main/RSP_CNN_dohyeon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 블로그

https://medium.com/@msmapark2/vgg16-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-very-deep-convolutional-networks-for-large-scale-image-recognition-6f748235242a

https://blog.naver.com/2hannseok/223229158970

https://diseny.tistory.com/entry/%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%ACconfusion-matrix?category=906035


In [ ]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#필요한거 import
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import numpy as np
import os

In [ ]:
base_dir = '/content/drive/MyDrive/RPS'  # 필요시 경로 수정
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# 이미지 크기 128x128로 설정 (100x100도 해봤는데 정확도 30퍼나옴..)
img_size = (128, 128)

# 데이터 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#generator 설정(_g)
train_g = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=16,  # 배치 크기 16으로 함
    class_mode='categorical'
)

val_g = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical'
)

Found 1950 images belonging to 3 classes.
Found 270 images belonging to 3 classes.


In [ ]:
# 데이터셋에 맞게 스텝 조정
epochstep = train_g.samples // train_g.batch_size
val_step = val_g.samples // val_g.batch_size

# VGG16 모델 사용
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

#모든 레이어 freeze
for layer in vgg_base.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
model = Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-5),  # 학습률을 줄임
    metrics=['accuracy']
)

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/RPS/model_checkpoint.keras', save_best_only=True, monitor='val_loss')

history = model.fit(
    train_g,
    steps_per_epoch=epochstep,
    epochs=10,
    validation_data=val_g,
    validation_steps=val_step,
    callbacks=[early_stopping, checkpoint]
)


# 혼동 행렬 계산
Y_pred = model.predict(val_g, val_g.samples // val_g.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

cm = confusion_matrix(val_g.classes, y_pred)
print(cm)


Epoch 1/10
121/121 [==============================] - 37s 299ms/step - loss: 1.1587 - accuracy: 0.4105 - val_loss: 0.8617 - val_accuracy: 0.6875
Epoch 2/10
121/121 [==============================] - 22s 176ms/step - loss: 0.9142 - accuracy: 0.5760 - val_loss: 0.7235 - val_accuracy: 0.7891
Epoch 3/10
121/121 [==============================] - 22s 181ms/step - loss: 0.7848 - accuracy: 0.6587 - val_loss: 0.6299 - val_accuracy: 0.8281
Epoch 4/10
121/121 [==============================] - 21s 172ms/step - loss: 0.7163 - accuracy: 0.7208 - val_loss: 0.5655 - val_accuracy: 0.8555
Epoch 5/10
121/121 [==============================] - 22s 180ms/step - loss: 0.6505 - accuracy: 0.7554 - val_loss: 0.5261 - val_accuracy: 0.8555
Epoch 6/10
121/121 [==============================] - 23s 187ms/step - loss: 0.5901 - accuracy: 0.7813 - val_loss: 0.4647 - val_accuracy: 0.8906
Epoch 7/10
121/121 [==============================] - 21s 176ms/step - loss: 0.5532 - accuracy: 0.8020 - val_loss: 0.4305 - val_ac

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import numpy as np
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
base_dir = '/content/drive/MyDrive/RPS'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# 이미지 크기 및 데이터 증강
img_size = (128, 128)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 데이터 생성기 설정
train_g = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,  # 배치 크기 증가
    class_mode='categorical'
)

val_g = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

# VGG16 모델 설정
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
for layer in vgg_base.layers:
    layer.trainable = False

# 모델 구성
model = Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))  # 은닉층 크기 증가
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# 모델 컴파일
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-4),  # 학습률 조정
    metrics=['accuracy']
)

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/RPS/model_checkpoint.keras', save_best_only=True, monitor='val_loss')

# 모델 학습
history = model.fit(
    train_g,
    steps_per_epoch=train_g.samples // train_g.batch_size,
    epochs=50,  # 에포크 증가
    validation_data=val_g,
    validation_steps=val_g.samples // val_g.batch_size,
    callbacks=[early_stopping, checkpoint]
)

# 혼동 행렬 계산
Y_pred = model.predict(val_g, val_g.samples // val_g.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(val_g.classes, y_pred)
print(cm)


Mounted at /content/drive
Found 1950 images belonging to 3 classes.
Found 270 images belonging to 3 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/50
60/60 [==============================] - 602s 10s/step - loss: 0.8784 - accuracy: 0.5923 - val_loss: 0.4795 - val_accuracy: 0.8125
Epoch 2/50
60/60 [==============================] - 21s 348ms/step - loss: 0.5553 - accuracy: 0.7737 - val_loss: 0.3157 - val_accuracy: 0.9023
Epoch 3/50
60/60 [==============================] - 21s 347ms/step - loss: 0.4674 - accuracy: 0.8227 - val_loss: 0.2970 - val_accuracy: 0.8867
Epoch 4/50
60/60 [==============================] - 21s 343ms/step - loss: 0.3975 - accuracy: 0.8472 - val_loss: 0.2405 - val_accuracy: 0.9375
Epoch 5/50
60/60 [==============================] - 22s 352ms/step - loss: 0.3576 - accuracy: 0.8655 - val_loss: 0.2411 - val_accuracy: 0.9141
Epoch 6/50
60/60 [==============================] - 20s 339ms/step - loss: 0.3611 - accuracy: 0.8577 - val_loss:

In [ ]:
from sklearn.metrics import accuracy_score

# 테스트 데이터 정확도 계산
accuracy = accuracy_score(val_g.classes, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 32.59%


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
base_dir = '/content/drive/MyDrive/RPS'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# 이미지 크기 및 데이터 증강
img_size = (128, 128)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 데이터 생성기 설정
train_g = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

val_g = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

# VGG16 모델 설정
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
for layer in vgg_base.layers:
    layer.trainable = False

# 모델 구성
model = Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(3, activation='softmax'))

# 모델 컴파일
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/RPS/model_checkpoint.keras', save_best_only=True, monitor='val_loss')

# 모델 학습
history = model.fit(
    train_g,
    steps_per_epoch=train_g.samples // train_g.batch_size,
    epochs=50,
    validation_data=val_g,
    validation_steps=val_g.samples // val_g.batch_size,
    callbacks=[early_stopping, checkpoint]
)

# 혼동 행렬 계산
Y_pred = model.predict(val_g, val_g.samples // val_g.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(val_g.classes, y_pred)
print("Confusion Matrix:")
print(cm)

# 테스트 데이터 정확도 계산
accuracy = accuracy_score(val_g.classes, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1950 images belonging to 3 classes.
Found 270 images belonging to 3 classes.
Epoch 1/50
60/60 [==============================] - 24s 379ms/step - loss: 1.2825 - accuracy: 0.3670 - val_loss: 0.9402 - val_accuracy: 0.6445
Epoch 2/50
60/60 [==============================] - 24s 396ms/step - loss: 1.1628 - accuracy: 0.4176 - val_loss: 0.8225 - val_accuracy: 0.7422
Epoch 3/50
60/60 [==============================] - 23s 376ms/step - loss: 1.0209 - accuracy: 0.5005 - val_loss: 0.7322 - val_accuracy: 0.7773
Epoch 4/50
60/60 [==============================] - 24s 392ms/step - loss: 0.9507 - accuracy: 0.5308 - val_loss: 0.6734 - val_accuracy: 0.8047
Epoch 5/50
60/60 [==============================] - 23s 381ms/step - loss: 0.8969 - accuracy: 0.5746 - val_loss: 0.6123 - val_accuracy: 0.8555
Epoch 6/50
60/60 [==============================] - 23s 382ms/step - loss

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
base_dir = '/content/drive/MyDrive/RPS'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# 이미지 크기 및 데이터 증강
img_size = (128, 128)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 데이터 생성기 설정
train_g = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

val_g = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

# ResNet50 모델 설정
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
for layer in resnet_base.layers:
    layer.trainable = False

# 모델 구성
model = Sequential()
model.add(resnet_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(3, activation='softmax'))

# 모델 컴파일
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/RPS/model_checkpoint.keras', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)

# 모델 학습
history = model.fit(
    train_g,
    steps_per_epoch=train_g.samples // train_g.batch_size,
    epochs=50,
    validation_data=val_g,
    validation_steps=val_g.samples // val_g.batch_size,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

# 혼동 행렬 계산
Y_pred = model.predict(val_g, val_g.samples // val_g.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(val_g.classes, y_pred)
print("Confusion Matrix:")
print(cm)

# 테스트 데이터 정확도 계산
accuracy = accuracy_score(val_g.classes, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


MessageError: Error: credential propagation was unsuccessful